In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

C:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
batch_size = 32 
num_classes = 10
epochs = 100
data_augmentation = True

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 163s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(80, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.25))


model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

C:\Anaconda\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 48)        13872     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 48)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 48)        0         
                                                        

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test), 
                        callbacks=[tbCallBack])

Using real-time data augmentation.


C:\Users\Parth Deshmukh\AppData\Local\Temp\ipykernel_4468\241606229.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),


Epoch 1/100
1562/1562 [==============================] - 141s 89ms/step - loss: 2.0394 - accuracy: 0.2285 - val_loss: 1.7876 - val_accuracy: 0.3550
Epoch 2/100
1562/1562 [==============================] - 113s 72ms/step - loss: 1.7701 - accuracy: 0.3398 - val_loss: 1.6011 - val_accuracy: 0.4254
Epoch 3/100
1562/1562 [==============================] - 109s 70ms/step - loss: 1.6491 - accuracy: 0.3900 - val_loss: 1.5017 - val_accuracy: 0.4540
Epoch 4/100
1562/1562 [==============================] - 109s 70ms/step - loss: 1.5760 - accuracy: 0.4214 - val_loss: 1.5030 - val_accuracy: 0.4518
Epoch 5/100
1562/1562 [==============================] - 107s 69ms/step - loss: 1.5169 - accuracy: 0.4474 - val_loss: 1.4213 - val_accuracy: 0.4733
Epoch 6/100
1562/1562 [==============================] - 107s 69ms/step - loss: 1.4642 - accuracy: 0.4644 - val_loss: 1.3487 - val_accuracy: 0.5060
Epoch 7/100
1562/1562 [==============================] - 110s 70ms/step - loss: 1.4142 - accuracy: 0.4840 - val_

1562/1562 [==============================] - 107s 69ms/step - loss: 0.7733 - accuracy: 0.7304 - val_loss: 0.6891 - val_accuracy: 0.7617
Epoch 57/100
1562/1562 [==============================] - 108s 69ms/step - loss: 0.7710 - accuracy: 0.7297 - val_loss: 0.6487 - val_accuracy: 0.7760
Epoch 58/100
1562/1562 [==============================] - 108s 69ms/step - loss: 0.7629 - accuracy: 0.7302 - val_loss: 0.7352 - val_accuracy: 0.7477
Epoch 59/100
1562/1562 [==============================] - 107s 69ms/step - loss: 0.7614 - accuracy: 0.7327 - val_loss: 0.6617 - val_accuracy: 0.7685
Epoch 60/100
1562/1562 [==============================] - 108s 69ms/step - loss: 0.7599 - accuracy: 0.7332 - val_loss: 0.6833 - val_accuracy: 0.7611
Epoch 61/100
1562/1562 [==============================] - 108s 69ms/step - loss: 0.7515 - accuracy: 0.7384 - val_loss: 0.6192 - val_accuracy: 0.7867
Epoch 62/100
1562/1562 [==============================] - 108s 69ms/step - loss: 0.7536 - accuracy: 0.7374 - val_loss: 

In [8]:
scores=model.evaluate(x_test,y_test,verbose=1)
print("Test Loss",scores[0])
print("Test Accuracy",scores[1])

313/313 [==============================] - 5s 17ms/step - loss: 0.5832 - accuracy: 0.8010
Test Loss 0.5831521153450012
Test Accuracy 0.8009999990463257
